## READ ME FIRST

This notebook produces a weekly release from the `develop` branch of Cytoscape.

The weekly release consists of Install4j generated installers that can be put on a public server to be downloaded by testers. This script will NOT push any code changes or resources to GitHub or Nexus or update any additional resources like the manual or API docs. 

This notebook is configured for the build server on which it is run. Notes have been made where local environment variables have been set.

Before starting this process, it is a good idea to Restart and Clear Output for this notebook's Kernel.


### 1. Set up Build Environment

This sets up the build environment, including Java and Maven versions, and the root directory of the build.


In [1]:
from subprocess import Popen, PIPE
import os
import shutil

Set the notebook directory. Note that this is system dependent.

In [2]:
NOTEBOOK_DIR = '/home/cybuilder/cytoscape-admin-scripts'

print(NOTEBOOK_DIR)

/home/cybuilder/cytoscape-admin-scripts


Set the Java environment. This is Java 11 for Cytoscape 3.8.0 and above.

Note that this is system dependent, and may need to be changed to reflect the system's configuration.

In [3]:
%env JAVA_HOME=/usr/lib/jvm/jre-11-openjdk

env: JAVA_HOME=/usr/lib/jvm/jre-11-openjdk


Set the MAVEN_HOME environment variable, as well as point the path to the correct Maven binaries. 

Note that this is system dependent and may need to be changed to reflect the system's configuration.

In [4]:
%env MAVEN_HOME=/opt/maven
%env PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin

env: MAVEN_HOME=/opt/maven
env: PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin


Prepare the target directory and clone the cytoscape git repo.

In [5]:
print('Changing to directory: ' + NOTEBOOK_DIR)
os.chdir(NOTEBOOK_DIR)

# Point to build location (the directory to clone parent cytoscape into)
BUILD_PARENT_DIR = os.path.join(os.getcwd(), 'release-build')
if not os.path.exists(BUILD_PARENT_DIR):
    os.mkdir(BUILD_PARENT_DIR)
else:
    shutil.rmtree(BUILD_PARENT_DIR)
    os.mkdir(BUILD_PARENT_DIR)

os.chdir(BUILD_PARENT_DIR)
![[ -d cytoscape ]] || git clone https://github.com/cytoscape/cytoscape
CYTOSCAPE_ROOT_DIR = os.path.join(BUILD_PARENT_DIR, 'cytoscape')
CYTOSCAPE_DIR = os.path.join(CYTOSCAPE_ROOT_DIR, 'cytoscape')

def cd(directory=BUILD_PARENT_DIR, *subdirs):
    if subdirs:
        directory = os.path.join(directory, *subdirs)
    if os.getcwd() != directory:
        os.chdir(directory)

Changing to directory: /home/cybuilder/cytoscape-admin-scripts
Cloning into 'cytoscape'...
remote: Enumerating objects: 577, done.
remote: Total 577 (delta 0), reused 0 (delta 0), pack-reused 577
Receiving objects: 100% (577/577), 758.90 KiB | 0 bytes/s, done.
Resolving deltas: 100% (284/284), done.


In [ ]:
STARTING_BRANCH = 'develop'     # develop for major release, release/3.X.X for minor release

## 2. Pull the develop branch of Cytoscape

Note that to execute the checkout step here, you will need to have set up an SSH key on this machine that does not use password validation, or else parts of the build will be stall when they require input. 

In [ ]:
cd(CYTOSCAPE_ROOT_DIR)
![[ -d cytoscape ]] || ./cy.sh init
cd(CYTOSCAPE_DIR)
!./cy.sh run-all "git checkout {STARTING_BRANCH}"

## 2a. Reset

In [ ]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git clean -f -d'
!./cy.sh run-all 'git reset --hard'

## 2. Verify status

In [ ]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git status'

## 3. Build Cytoscape and ensure no errors

This may take a while. Expect to build subrepos first before building from the root directory

Currently, you should expect two or so non-fatal javadoc-bundle-options related errors.

In [ ]:
cd(CYTOSCAPE_DIR)
with open('build_output.txt', 'w') as outf:
    process = Popen('mvn clean install -Dmaven.test.skip=true'.split(' '), 
                stdout=outf,
                cwd=CYTOSCAPE_DIR)
    process.wait()

print("Showing ERROR lines in build...")
!cat build_output.txt | grep ERROR

## 4. Build Cytoscape installers

This requires Install4J to be configured on your machine and for the Install4j project file to point to the correct Mac signing file. The keystore password is set below (blank in this config), to be interpreted by the Install4j Maven plugin.

In [ ]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
%env MAC_KEYSTORE_PASSWORD=
!mvn clean install -U

## 5. Copying Cytoscape installers to weekly download page

When completed installer executables can be found in `cytoscape/cytoscape/gui-distribution/packaging/target/media` and compressed builds for Linux and Windows can be found in `cytoscape/cytoscape/gui-distribution/assembly/target`

all of which should be copied to `/var/www/html/cytoscape-builds/Cytoscape-3.9.0/<DATE>`



In [ ]:
!mkdir -p /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/release-build/cytoscape/cytoscape/gui-distribution/packaging/target/media/* /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/release-build/cytoscape/cytoscape/gui-distribution/assembly/target/*.{gz,zip} /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)


## 6. Notarizing on idekerlab-macmini

The install4j generated .dmg must be notarized to run on macOS 10.15 and above. The .dmg will be sent to idekerlab-macmini and submitted for notarization. An email will be sent to William Markuske when the notarization process is complete. As soon as the .dmg is notarized it will work and no further action is needed.

In [ ]:
!rsync -av --progress /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)/*.dmg idekerlab@idekerlab-macmini.ucsd.edu:~/apps_to_notarize/
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizedmg_jing.sh ~/apps_to_notarize/*.dmg Snapshot.$(date +%Y%m%d)'    

You can check the status of the notarization by pasting in the `RequestUUID` value into the following command and running.

In [ ]:
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizestatus_jing.sh c1e8e12b-6499-4e8a-9b8f-0bcefea62e8e'